# 🎨 Spot the Artist - Anna Laurini Art  pupupu

This notebook runs the AI backend for verifying Anna Laurini's street art using your **A100 GPU**.

## Setup Instructions
1. Go to **Runtime → Change runtime type → A100 GPU**
2. Run all cells in order
3. Upload your reference images when prompted
4. Copy the ngrok URL to use with your frontend

## 🚀 A100 Performance
With your A100 GPU, you'll get:
- **~5x faster** model loading compared to T4
- **~10x faster** inference per image
- Verification in **under 50ms** per image


## Step 1: Check GPU & Install Dependencies


In [ ]:
# Check if GPU is available
import torch
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🚀 GPU: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    if "A100" in gpu_name:
        print("🔥 A100 detected - Maximum performance!")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → A100 GPU")


In [ ]:
# Install required packages
!pip install -q fastapi uvicorn python-multipart transformers Pillow pyngrok nest-asyncio


## Step 2: Upload Reference Images

Upload 15-20 images of Anna Laurini's artwork. These will be used to verify user uploads.

In [ ]:
import os
from google.colab import files

# Create reference art folder
os.makedirs('reference_art', exist_ok=True)

print("📤 Upload your reference images (Anna Laurini's artwork)")
print("   Supported formats: .jpg, .jpeg, .png, .webp")
print("   Recommended: 30-50 high-quality images for best accuracy\n")
print("💡 TIP: You can select multiple files at once in the file picker!\n")

uploaded = files.upload()

# Move uploaded files to reference_art folder
for filename in uploaded.keys():
    # Handle case where file might already exist
    dest = f'reference_art/{filename}'
    if os.path.exists(dest):
        base, ext = os.path.splitext(filename)
        dest = f'reference_art/{base}_copy{ext}'
    os.rename(filename, dest)
    print(f"✅ Saved: {os.path.basename(dest)}")

total = len([f for f in os.listdir('reference_art') if not f.startswith('.')])
print(f"\n📁 Total reference images: {total}")
if total >= 30:
    print("🎯 Great! 30+ images gives excellent accuracy.")
elif total >= 15:
    print("👍 Good baseline. Consider adding more for better accuracy.")


## Step 3: Create the CLIP Service

This is the AI brain that compares images using OpenAI's CLIP model.

In [ ]:
from pathlib import Path
from typing import Optional
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel


class CLIPService:
    """Service for verifying artwork using CLIP embeddings."""
    
    MODEL_NAME = "openai/clip-vit-base-patch32"
    SUPPORTED_EXTENSIONS = {".jpg", ".jpeg", ".png", ".webp"}
    
    def __init__(self, reference_dir: str = "reference_art"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🔧 Using device: {self.device}")
        
        # Load CLIP model
        print(f"📥 Loading CLIP model: {self.MODEL_NAME}")
        self.model = CLIPModel.from_pretrained(self.MODEL_NAME).to(self.device)
        self.processor = CLIPProcessor.from_pretrained(self.MODEL_NAME)
        self.model.eval()
        print("✅ CLIP model loaded successfully")
        
        # Store reference embeddings
        self.reference_embeddings: Optional[torch.Tensor] = None
        self.reference_names: list[str] = []
        
        # Load reference images
        self.reference_dir = Path(reference_dir)
        self._load_reference_images()
    
    def _load_reference_images(self) -> None:
        """Load and cache embeddings for all reference images."""
        if not self.reference_dir.exists():
            print(f"⚠️ Reference directory not found: {self.reference_dir}")
            return
        
        image_files = [
            f for f in self.reference_dir.iterdir()
            if f.suffix.lower() in self.SUPPORTED_EXTENSIONS
        ]
        
        if not image_files:
            print(f"⚠️ No reference images found in {self.reference_dir}")
            return
        
        print(f"📚 Loading {len(image_files)} reference images...")
        
        embeddings = []
        for img_path in image_files:
            try:
                embedding = self._get_image_embedding(img_path)
                embeddings.append(embedding)
                self.reference_names.append(img_path.name)
            except Exception as e:
                print(f"❌ Error loading {img_path.name}: {e}")
        
        if embeddings:
            self.reference_embeddings = torch.cat(embeddings, dim=0)
            print(f"✅ Loaded {len(embeddings)} reference embeddings")
    
    def _get_image_embedding(self, image_path: Path) -> torch.Tensor:
        image = Image.open(image_path).convert("RGB")
        return self._embed_image(image)
    
    def _embed_image(self, image: Image.Image) -> torch.Tensor:
        inputs = self.processor(images=image, return_tensors="pt").to(self.device)
        with torch.no_grad():
            image_features = self.model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        return image_features
    
    def verify_image(self, image: Image.Image) -> dict:
        """Verify if an image matches Anna Laurini's artwork.
        Uses top-k matching for more robust verification."""
        if self.reference_embeddings is None or len(self.reference_embeddings) == 0:
            return {
                "is_verified": False,
                "confidence": 0.0,
                "message": "No reference images loaded.",
                "best_match": None
            }
        
        query_embedding = self._embed_image(image)
        similarities = torch.mm(query_embedding, self.reference_embeddings.t()).squeeze(0)
        
        # Best match
        best_similarity, best_idx = similarities.max(dim=0)
        best_similarity = best_similarity.item()
        best_match = self.reference_names[best_idx.item()]
        
        # Use top-k average for robust scoring
        # Scale k based on number of references: ~10%, min 3, max 10
        k = max(3, min(10, len(self.reference_names) // 10 + 1))
        top_k_similarities, _ = similarities.topk(k)
        avg_top_k = top_k_similarities.mean().item()
        
        confidence = self._scale_similarity(avg_top_k)
        
        # Stricter thresholds to reduce false positives
        if confidence >= 80:
            is_verified = True
            message = "✅ Verified! This looks like Anna Laurini's artwork!"
        elif confidence >= 60:
            is_verified = False
            message = "🤔 Uncertain. This might be Anna Laurini's art."
        else:
            is_verified = False
            message = "❌ Not recognized as Anna Laurini's artwork."
        
        return {
            "is_verified": is_verified,
            "confidence": round(confidence, 1),
            "message": message,
            "best_match": best_match,
            "raw_similarity": round(best_similarity, 4),
            "avg_top_k": round(avg_top_k, 4)
        }
    
    def _scale_similarity(self, similarity: float) -> float:
        """Scale similarity to 0-100% with proper capping."""
        similarity = max(0.0, min(1.0, similarity))
        
        if similarity >= 0.80:
            score = 90 + (similarity - 0.80) / 0.20 * 10
            return min(100.0, score)  # Cap at 100%
        elif similarity >= 0.70:
            return 75 + (similarity - 0.70) / 0.10 * 15
        elif similarity >= 0.55:
            return 50 + (similarity - 0.55) / 0.15 * 25
        elif similarity >= 0.40:
            return 25 + (similarity - 0.40) / 0.15 * 25
        else:
            return similarity / 0.40 * 25
    
    def get_reference_count(self) -> int:
        return len(self.reference_names)


# Initialize the service
print("\n" + "="*50)
clip_service = CLIPService("reference_art")
print("="*50)


## Step 4: Test the Model (Optional)

Upload a test image to verify everything works before starting the server.
 

In [ ]:
# Optional: Test with an image
from google.colab import files
from PIL import Image
import io

print("📤 Upload a test image to verify:")
test_upload = files.upload()

for filename, data in test_upload.items():
    image = Image.open(io.BytesIO(data)).convert("RGB")
    result = clip_service.verify_image(image)
    
    print(f"\n🖼️ Testing: {filename}")
    print(f"   Verified: {result['is_verified']}")
    print(f"   Confidence: {result['confidence']}%")
    print(f"   Best match: {result['best_match']}")
    print(f"   Message: {result['message']}")


## Step 5: Create the FastAPI Server


In [ ]:
import io
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from PIL import Image

# Create FastAPI app
app = FastAPI(
    title="Anna Laurini Art Verification API",
    description="AI-powered verification of Anna Laurini street art",
    version="1.0.0"
)

# Enable CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class VerificationResponse(BaseModel):
    is_verified: bool
    confidence: float
    message: str
    best_match: str = None


@app.get("/api/health")
async def health_check():
    return {
        "status": "healthy",
        "reference_images_loaded": clip_service.get_reference_count(),
        "device": clip_service.device
    }


@app.post("/api/verify", response_model=VerificationResponse)
async def verify_artwork(file: UploadFile = File(...)):
    """Verify if an uploaded image matches Anna Laurini's artwork."""
    
    if not file.content_type or not file.content_type.startswith("image/"):
        raise HTTPException(status_code=400, detail="Please upload an image file.")
    
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")
        result = clip_service.verify_image(image)
        
        return VerificationResponse(
            is_verified=result["is_verified"],
            confidence=result["confidence"],
            message=result["message"],
            best_match=result.get("best_match")
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing image: {str(e)}")


print("✅ FastAPI app created!")


## Step 6: Start the Server with ngrok

### Setup (one-time):
1. Go to [ngrok.com](https://ngrok.com) and create a free account
2. Go to [dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken) → Copy your **authtoken**
3. Go to [dashboard.ngrok.com/domains](https://dashboard.ngrok.com/domains) → Click **"Create Domain"** → Copy your **domain** (e.g., `your-name-abc123.ngrok-free.app`)
4. Paste both values in the cell below


# This cell intentionally left empty - placeholder for ngrok token cell above


In [ ]:
# Enter your ngrok credentials here
NGROK_AUTH_TOKEN = ""  # <-- Paste your authtoken here!
NGROK_DOMAIN = ""      # <-- Paste your domain here (e.g., "your-name-abc123.ngrok-free.app")

if not NGROK_AUTH_TOKEN:
    print("⚠️ Missing authtoken!")
    print("   Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
elif not NGROK_DOMAIN:
    print("⚠️ Missing domain!")
    print("   1. Go to: https://dashboard.ngrok.com/domains")
    print("   2. Click 'Create Domain' (it's free)")
    print("   3. Copy the domain (e.g., your-name-abc123.ngrok-free.app)")
else:
    print("✅ ngrok configured!")
    print(f"   Token: {NGROK_AUTH_TOKEN[:10]}...")
    print(f"   Domain: {NGROK_DOMAIN}")


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Apply nest_asyncio to allow running uvicorn in Colab
nest_asyncio.apply()

# Configure ngrok with your authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel with your static domain
# This is required for ngrok free tier (as of 2024)
public_url = ngrok.connect(8000, domain=NGROK_DOMAIN)
public_url_str = f"https://{NGROK_DOMAIN}"

print("\n" + "="*60)
print("🚀 SERVER IS RUNNING!")
print("="*60)
print(f"\n🌐 Public URL: {public_url_str}")
print(f"\n📋 API Endpoints:")
print(f"   • Health check: {public_url_str}/api/health")
print(f"   • Verify art:   {public_url_str}/api/verify")
print(f"   • API docs:     {public_url_str}/docs")
print(f"\n💡 To use with the React frontend:")
print(f"   Set VITE_API_URL={public_url_str}")
print("\n" + "="*60)
print("⚠️ Keep this cell running! The server stops when you stop it.")
print("="*60 + "\n")

# Run the server (this blocks until interrupted)
uvicorn.run(app, host="0.0.0.0", port=8000)


## 🎉 Done!

Your API is now running. You can:

1. **Test the API** - Open the `/docs` URL in your browser to use the Swagger UI
2. **Connect your frontend** - Update your React app to use the ngrok URL

### To connect the React frontend:

Create a `.env` file in the `frontend/` folder:
```
VITE_API_URL=https://xxxx-xx-xx-xx-xx.ngrok-free.app
```

Then run:
```bash
cd frontend
npm run dev
```
